In [60]:
import random
import numpy as np

In [61]:
class Set:
    def __init__(self, players, n = 10, winBonus = 10, roundBonus = 2, ordered = False, printMode = True):
        '''
        n (int) - number of rounds to be played per set
        players (list) - list of k player objects
        winBonus (int) - bonus awarded to the player who "takes" and therefore wins the Set
        roundBonus (int) - incremental bonus awarded for every round the set lasts
        k (int) - number of players for the set
        printMode (bool) - testing bool - used to print stuff out while game is going to make it look good
        
        
        Initialized later:
        
        playHist (k x n numpy array) - records actions for each player at every played round
        set_scores (k x 1 numpy array) - contains the calculated score for each player after set is played
        winner (int) - index of the player who "takes" and wins the set
        r_end (int) - round where a player decided to "take"
        '''
        self.n = n
        self.players = players
        self.winBonus = winBonus
        self.roundBonus = roundBonus
        self.k = len(players)
        self.printMode = printMode
        
        # shuffle player order if not specified
        if not ordered:
            random.shuffle(self.players)
            
        
    def initialize_playHistory(self):
        '''
        Initialize the k x n playHistory array that will record the action in round r for player i in row i column r
        
        All actions that do not happen as a result of the game ending are NaN  -- (playHist)
        
        Also initializes each player's index for the set - index is used as row i in playHistory and set_scores
        
        Used in -play-
        '''
        self.playHist = np.empty((self.k, self.n))
        self.playHist[:] = np.NaN
        
        # initialize row/index for each player
        for idx, player in enumerate(self.players):
            player.idx = idx
            
            
    def record_actions(self, r):
        '''
        Records the action for each player in the playHistory array for one round. A 0 for row i column r represents
        a decision of pass for player i while a 1 represents a Take.
        
        Used in -play-
        '''
        
        for player in self.players:
            decision = player.manual_decision(r = r, n = self.n, k = self.k)
            if(decision == 'Pass'):
                self.playHist[player.idx][r] = 0
            elif(decision == 'Take'):
                self.playHist[player.idx][r] = 1
                return player.idx
            
        return None
    
        
    def play(self):
        '''
        Plays through the set and records winner and r_end
        '''
        self.initialize_playHistory()
        
        # play through this n rounds
        for r in range(self.n):
            
            if(self.printMode):
                print("==============================")
            
            result = self.record_actions(r)
            
            # result will return as an integer of the player's index who decided to "take", or None if all players "pass"
            if(isinstance(result, int)):
                self.winner = result
                self.r_end = r
                return
            
            # next round begins if no one decided to "take"
            
        # no one decided to "take" so r_end and winner do not exist
        self.winner = np.nan
        self.r_end = np.nan
        return
    
    
    def calculate_score(self):
        '''
        Returns a k x 1 array of the payoff for each player i -- (set_scores)
        '''
        self.set_scores = np.zeros((self.k, 1))
        
        # multiply number of rounds completed (r_end) by round bonus, r_end will be nan if no one "takes" 
        if(~np.isnan(self.r_end)):
            self.set_scores += self.r_end * self.roundBonus
        else:
            self.set_scores += self.n * self.roundBonus
            
        if(~np.isnan(self.winner)):
            self.set_scores[self.winner] += self.winBonus
            
        return      
    

In [62]:
class Player:
    def __init__(self):
        '''
        idx (int) - the index assigned to this player during a set, used to match the player to a row in the records
        '''
        self.idx = np.nan
       
    def manual_decision(self, r, n, k):
        '''
        User manually inputs the decision to "take" or "pass", 1 is "take", 0 is "pass"
        User is given the history of plays, round number, total of rounds, total number of players
        '''
        print('Round {} of {}'.format(r + 1, n))
        print('You are in position {} out of {}'.format(self.idx + 1, k))
        print()
        
        decision = input('1 is Take, 0 is Pass: ')
        print()
        while True:
            if decision == '0':
                return 'Pass'
            elif decision == '1':
                return 'Take'
            else:
                decision = input('Error: 1 is Take, 0 is Pass: ')
                print()
      

In [63]:
def tester():
    alex = Player()
    jon = Player()
    steve = Player()
    theboys = [alex, jon, steve]
    game = Set(theboys)
    game.play()
    game.calculate_score()
    print('Game Play History')
    print(game.playHist)
    print()
    print('Game Scores')
    print(game.set_scores)
    
    return 

In [64]:
tester()

Round 1 of 10
You are in position 1 out of 3

1 is Take, 0 is Pass: 0

Round 1 of 10
You are in position 2 out of 3

1 is Take, 0 is Pass: 0

Round 1 of 10
You are in position 3 out of 3

1 is Take, 0 is Pass: 0

Round 2 of 10
You are in position 1 out of 3

1 is Take, 0 is Pass: 0

Round 2 of 10
You are in position 2 out of 3

1 is Take, 0 is Pass: 0

Round 2 of 10
You are in position 3 out of 3

1 is Take, 0 is Pass: 0

Round 3 of 10
You are in position 1 out of 3

1 is Take, 0 is Pass: 0

Round 3 of 10
You are in position 2 out of 3

1 is Take, 0 is Pass: 0

Round 3 of 10
You are in position 3 out of 3

1 is Take, 0 is Pass: 0

Round 4 of 10
You are in position 1 out of 3

1 is Take, 0 is Pass: 0

Round 4 of 10
You are in position 2 out of 3

1 is Take, 0 is Pass: 1

Game Play History
[[ 0.  0.  0.  0. nan nan nan nan nan nan]
 [ 0.  0.  0.  1. nan nan nan nan nan nan]
 [ 0.  0.  0. nan nan nan nan nan nan nan]]

Game Scores
[[ 6.]
 [16.]
 [ 6.]]
